# Talk to Graph

In [2]:
from neo4j import GraphDatabase
import openai
import langchain
import os

## Connection to Neo4j

In [9]:
# Verbindungsinformationen für Neo4j-Datenbank
uri = "bolt://localhost:7687"  
user = "neo4j"                  
password = ""         

# Verbindung zur Neo4j-Datenbank herstellen
driver = GraphDatabase.driver(uri, auth=(user, password))


In [10]:
def execute_query(query):
    with driver.session() as session:  
        result = session.run(query)
        for record in result:
            print(record)  

# Testabfrage, um alle Knoten zu zählen
test_query = "MATCH (n) RETURN COUNT(n) as node_count"
execute_query(test_query)

<Record node_count=20>


## Graph Abfragen

In [85]:
def get_transaction_between_persons(person_id4, person_id12):
    with driver.session() as session:
        result = session.run("""
        MATCH (p1:Person {id: $person_id4})-[:TRANSACTION]->(p2:Person {id: $person_id12})
        RETURN count(*) > 0 as transaction_exists
        """, person_id4=person_id4, person_id12=person_id12)
        return result.single()[0]


In [75]:
def get_most_connected_person():
    with driver.session() as session:
        result = session.run("""
        MATCH (p:Person)-[:TRANSACTION]-()
        RETURN p.name AS person, count(*) AS connections
        ORDER BY connections DESC
        LIMIT 1
        """)
        return result.single()

## Fragebeispiele

In [86]:
# Fragebeispiel: Hat Person 4 mit Person 12 Transaktionen?
transaction_exists = get_transaction_between_persons(4, 12)
print(f"Transaction exists: {transaction_exists}")

Transaction exists: True


#### Fragebeispiel: Welche Person hat mit den meisten anderen Personen Transaktionen durchgeführt?

In [76]:
# Fragebeispiel: Welche Person hat mit den meisten anderen Personen Transaktionen durchgeführt?
most_connected_person = get_most_connected_person()
print(f"Most connected person: {most_connected_person['person']} with {most_connected_person['connections']} connections")

Most connected person: Isabella Hoffmann with 6 connections


In [63]:
def find_transactions(person_id1, person_id2):
    with driver.session() as session:
        query = """
        MATCH (p1:Person)-[t:TRANSACTION]->(p2:Person)
        WHERE p1.id = $person_id1 AND p2.id = $person_id2
        RETURN t
        """
        result = session.run(query, person_id1=person_id1, person_id2=person_id2)
        return [record["t"] for record in result]

transactions = find_transactions(1, 9)

if transactions:
    print(f"Es gibt {len(transactions)} Transaktion(en) zwischen Person 1 und Person 9.")
    for transaction in transactions:
        print(f"Transaktion: {transaction}")
else:
    print("Es gibt keine Transaktionen zwischen Person 1 und Person 9.")

Es gibt 1 Transaktion(en) zwischen Person 1 und Person 9.
Transaktion: <Relationship element_id='5:7e939ad8-f72d-4ca3-b7e8-1d104406237e:8' nodes=(<Node element_id='4:7e939ad8-f72d-4ca3-b7e8-1d104406237e:40' labels=frozenset() properties={}>, <Node element_id='4:7e939ad8-f72d-4ca3-b7e8-1d104406237e:6' labels=frozenset() properties={}>) type='TRANSACTION' properties={'amount': 8455.0, 'currency': 'GBP'}>


## OpenAI und LangChain integrieren

In [3]:
openai.api_key = os.getenv("OPENAI_API_KEY", "")

In [5]:
import openai

class MyOpenAIWrapper:
    def __init__(self, api_key):
        self.api_key = api_key
        openai.api_key = api_key

    def generate(self, prompt, model="gpt-3.5-turbo"):
        response = openai.ChatCompletion.create(
            model=model,
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": prompt}
            ]
        )
        return response['choices'][0]['message']['content']

api_key = ""
openai_wrapper = MyOpenAIWrapper(api_key=api_key)

prompt = "What are the main benefits of using LangChain?"
response = openai_wrapper.generate(prompt)
print(response)


RateLimitError: You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.

In [23]:
# Schritt 1: Neo4j-Daten abfragen
person_id = 1  # Beispiel-ID
query = "MATCH (p:Person {id: $person_id})-->(r:Relationship) RETURN r"
with driver.session() as session:
    result = session.run(query, person_id=person_id)
    relationships = [record["r"] for record in result]

# Schritt 2: Abfrageergebnisse als Prompt vorbereiten
prompt = f"Erkläre die Beziehungen von Person {person_id}: " + ", ".join([rel["name"] for rel in relationships])

# Schritt 3: LLM für die Generierung der Antwort nutzen
response = openai_wrapper.generate(prompt)  # Angenommen, openai_wrapper ist bereits definiert

# Schritt 4: Antwort verarbeiten und zurückgeben
print(response)


Um die Beziehungen von Person 1 zu erklären, benötige ich zusätzliche Informationen. Welche Beziehungen von Person 1 interessieren Sie? Sind Sie auf der Suche nach Informationen zu Freunden, Familie, Arbeitskollegen oder anderen Kontakten? Bitte geben Sie mir mehr Details, damit ich Ihnen besser weiterhelfen kann.


In [65]:
def get_relationship_info(person_id1, person_id2):
    query = """
    MATCH (p1:Person {id: $person_id1})-[r:TRANSACTION]->(p2:Person {id: $person_id2})
    RETURN r.amount as amount, r.currency as currency
    """
    with driver.session() as session:
        result = session.run(query, person_id1=person_id1, person_id2=person_id2)
        for record in result:
            # Angenommen, jede Transaktion hat einen Betrag und eine Währung
            return {"amount": record["amount"], "currency": record["currency"]}
    return None


In [67]:
def create_prompt_for_llm(relationship_info):
    if relationship_info:
        prompt = f"Explain the significance of a {relationship_info['amount']} {relationship_info['currency']} transaction between two individuals."
    else:
        prompt = "There's no transaction between these individuals. Could you hypothesize reasons why two individuals might not have transactions?"
    return prompt


In [68]:
response = openai_wrapper.generate(create_prompt_for_llm(get_relationship_info(1, 9)))
print(response)

A transaction of 8455.0 GBP between two individuals could have various significances depending on the context of the transaction. Here are a few possibilities:

1. It could be a payment for goods or services exchanged between the two individuals. The transaction amount of 8455.0 GBP could reflect the value of the items or services being exchanged.

2. It could be a gift or a loan between the individuals. The specific amount of 8455.0 GBP may have been chosen for a particular reason or based on an agreement between the two individuals.

3. It could be a regular payment, such as rent or a monthly allowance, that one individual owes to the other. The amount of 8455.0 GBP could indicate the agreed-upon payment for a specific period.

4. It could also be a transferring of funds between family members or friends for personal reasons, such as helping out in times of need or for a celebration.

Overall, the significance of the transaction would depend on the relationship and agreement between 

In [88]:
from neo4j import GraphDatabase
import openai

class MyOpenAIWrapper:
    def __init__(self, api_key):
        self.api_key = api_key
        openai.api_key = api_key

    def generate(self, prompt, model="gpt-3.5-turbo"):
        response = openai.ChatCompletion.create(
            model=model,
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": prompt}
            ]
        )
        return response['choices'][0]['message']['content']

class Neo4jConnection:
    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        self.driver.close()

    def query(self, query, parameters=None, db=None):
        with self.driver.session(database=db) as session:
            result = session.run(query, parameters)
            return [record for record in result]

# Initialisierung
uri = "bolt://localhost:7687"
user = "neo4j"
password = ""
neo4j_conn = Neo4jConnection(uri, user, password)
api_key = ""
openai_wrapper = MyOpenAIWrapper(api_key=api_key)

# Beispiel 1: Prüfen, ob eine Transaktion zwischen zwei Personen besteht
def check_transaction_between(person_id4, person_id12):
    query = f"""
    MATCH (p1:Person {{id: {person_id4}}})-[t:TRANSACTION]->(p2:Person {{id: {person_id12}}})
    RETURN EXISTS((p1)-[t]->(p2)) AS transactionExists
    """
    result = neo4j_conn.query(query)
    exists = result[0]["transactionExists"]
    prompt = f"Discuss the implications of a transaction existing or not existing between two individuals, specifically between individuals {person_id4} and {person_id12}."
    response = openai_wrapper.generate(prompt)
    print(f"Transaction exists: {exists}")
    print(f"LLM Response: {response}")

# Beispiel 2: Identifizieren der Person mit den meisten Transaktionen
def get_most_connected_person():
    query = """
    MATCH (p:Person)-[t:TRANSACTION]->()
    RETURN p.id AS personID, COUNT(t) AS numberOfTransactions
    ORDER BY numberOfTransactions DESC
    LIMIT 1
    """
    result = neo4j_conn.query(query)
    person_id, connections = result[0]["personID"], result[0]["numberOfTransactions"]
    prompt = f"What role might the person with the most transactions play within their network? Person ID: {person_id} has the highest number of transactions: {connections}."
    response = openai_wrapper.generate(prompt)
    print(f"Most connected person: Person {person_id} with {connections} connections")
    print(f"LLM Response: {response}")

# Führen Sie die Funktionen aus, um die Ergebnisse zu sehen
check_transaction_between(4, 12)
get_most_connected_person()

# Vergessen Sie nicht, die Verbindung zu schließen, wenn Sie fertig sind
neo4j_conn.close()


Transaction exists: True
LLM Response: The implications of a transaction existing or not existing between individuals 4 and 12 can vary depending on the context of the transaction. Here are some potential implications:

If a transaction exists between individuals 4 and 12:
1. Economic implications: A transaction between individuals 4 and 12 could indicate that they are engaged in some form of economic exchange, such as buying and selling goods or services. This could impact their financial well-being and contribute to economic activity.

2. Relationship implications: The presence of a transaction between individuals 4 and 12 may indicate a level of interaction and relationship between them. It could suggest that they have a mutual understanding or agreement that involves exchanging something of value.

3. Legal implications: Depending on the nature of the transaction, there could be legal implications involved, such as contracts, obligations, and potential legal consequences if either 

In [82]:
suspicious_transactions = [
    {"amount": 100000, "timestamp": "2023-01-02T03:45:00", "personID": 4},
    # Weitere verdächtige Transaktionen...
]

prompt = "Given a series of transactions that occurred at unusual hours and involved unusually high amounts, what patterns might indicate fraudulent activity? Here are some examples: " + ", ".join([f"{t['amount']} at {t['timestamp']}" for t in suspicious_transactions])

response = openai_wrapper.generate(prompt)
print(f"LLM Insight on Fraud Detection: {response}")


LLM Insight on Fraud Detection: Several patterns that might indicate fraudulent activity based on the provided example of a transaction amount of 100,000 at 03:45:00 on January 2, 2023, are as follows:

1. Unusually high transaction amount: A transaction amount of 100,000 is significantly higher than typical transaction amounts for most individuals or businesses. Such a high amount could indicate potential fraud, especially if it is not in line with the customer's usual spending patterns.

2. Unusual transaction timing: The transaction occurring at 03:45:00 indicates an unusual hour for a transaction. Fraudsters may choose to perform transactions during off-hours to avoid detection or to take advantage of lower monitoring activity during such times.

3. Lack of prior history: If the transaction amount or timing is inconsistent with the customer's historical transaction data, it could be a red flag for potential fraud. A sudden spike in transaction amounts or activity without any previo

In [1]:
from neo4j import GraphDatabase

class Neo4jConnection:
    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))

    def find_large_transactions(self):
        query = """
        MATCH (p4:Person)-[t:TRANSACTION]->(p12:Person)
        WHERE t.amount > 10000
        RETURN p4.id AS from, p12.id AS to, t.amount AS amount, t.currency AS currency
        """
        transactions = []
        with self.driver.session() as session:
            result = session.run(query)
            for record in result:
                transactions.append({
                    "from": record["from"],
                    "to": record["to"],
                    "amount": record["amount"],
                    "currency": record["currency"]
                })
        return transactions

    def find_users_with_multiple_transactions(self):
        query = """
        MATCH (p:Person)-[t:TRANSACTION]->()
        WITH p, count(t) AS transactions
        WHERE transactions > 3
        RETURN p.id AS personID, transactions
        """
        users = []
        with self.driver.session() as session:
            result = session.run(query)
            for record in result:
                users.append({
                    "personID": record["personID"],
                    "transactions": record["transactions"]
                })
        return users
    


class MyOpenAIWrapper:
    def __init__(self, api_key):
        self.api_key = api_key

    def generate(self, prompt, model="gpt-3.5-turbo"):
        import openai
        openai.api_key = self.api_key
        try:
            response = openai.ChatCompletion.create(
                model=model,
                messages=[{"role": "system", "content": "You are a helpful assistant analyzing transactions for potential fraud."},
                          {"role": "user", "content": prompt}]
            )
            return response.choices[0].message['content']  
        except Exception as e:
            print(f"Error: {e}")
            return "An error occurred while generating the response."

# Initialize Neo4j and OpenAI connections
neo4j_conn = Neo4jConnection("bolt://localhost:7687", "neo4j", "")
openai_wrapper = MyOpenAIWrapper(api_key="")

# Find suspicious transactions and analyze them with LLM
large_transactions = neo4j_conn.find_large_transactions()
for transaction in large_transactions:
    prompt = f"Analyze the transaction from {transaction['from']} to {transaction['to']} with amount {transaction['amount']} {transaction['currency']}. Could this indicate fraudulent activity?"
    response = openai_wrapper.generate(prompt)
    print(f"LLM Analysis for transaction {transaction['from']} to {transaction['to']}: {response}")

# Find users with multiple transactions and analyze potential fraud
users_with_multiple_transactions = neo4j_conn.find_users_with_multiple_transactions()
for user in users_with_multiple_transactions:
    prompt = f"Person {user['personID']} has conducted {user['transactions']} transactions. Analyze for potential fraudulent behavior."
    response = openai_wrapper.generate(prompt)
    print(f"LLM Analysis for person {user['personID']}: {response}")




LLM Analysis for transaction 4 to 12: To determine if this transaction is potentially fraudulent, we need more information to understand the context. The transaction description "from 4 to 12" doesn't provide sufficient details to assess its legitimacy. 

Here are some additional questions to consider in evaluating the transaction:

1. Who are parties involved in the transaction?
2. What is the nature of the transaction (e.g., purchase, transfer, payment)?
3. Is this a known or common transaction for the parties involved?
4. Is the transaction amount typical for the parties involved?
5. Are there any red flags or anomalies associated with the transaction?

Without more information, it is challenging to definitively determine whether the transaction is fraudulent. It is recommended to investigate further and verify the details before drawing any conclusions.
LLM Analysis for transaction None to None: To determine if this transaction could indicate fraudulent activity, we would need more

## Neo4j Algorithmen für Übergabe an GPT

In [20]:
from neo4j import GraphDatabase

class Neo4jConnection:
    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))

    def calculate_page_rank(self):
        query = """
        CALL gds.pageRank.write('myGraphTest', {
            writeProperty: 'pageRank'
        })
        YIELD nodePropertiesWritten;
        """
        with self.driver.session() as session:
            result = session.run(query)
            return result.single()[0] 


class MyOpenAIWrapper:
    def __init__(self, api_key):
        self.api_key = api_key

    def generate(self, prompt, model="gpt-3.5-turbo"):
        import openai
        openai.api_key = self.api_key
        try:
            response = openai.ChatCompletion.create(
                model=model,
                messages=[{"role": "system", "content": "You are a helpful assistant analyzing transactions for potential fraud."},
                          {"role": "user", "content": prompt}]
            )
            return response.choices[0].message['content']
        except Exception as e:
            print(f"Error: {e}")
            return "An error occurred while generating the response."

# Initialize Neo4j and OpenAI connections
neo4j_conn = Neo4jConnection("bolt://localhost:7687", "neo4j", "")
openai_wrapper = MyOpenAIWrapper(api_key="")

# Calculate PageRank to identify key individuals
page_rank_message = neo4j_conn.calculate_page_rank()
print(page_rank_message)

# Example of analyzing the role of key individuals identified by PageRank
prompt = "Examine the role of individuals with high PageRank scores in the context of potential money laundering activities."
response = openai_wrapper.generate(prompt)
print(f"LLM Analysis: {response}")


43


Failed to write data to connection IPv4Address(('localhost', 7687)) (ResolvedIPv4Address(('127.0.0.1', 7687)))
Failed to write data to connection IPv4Address(('localhost', 7687)) (ResolvedIPv4Address(('127.0.0.1', 7687)))


Error: You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.
LLM Analysis: An error occurred while generating the response.
